<a href="https://colab.research.google.com/github/mariasald/avd-2024-01/blob/main/Atividade_Pr%C3%A1tica_4_O_Problema_do_Transiente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Alunos**

* Gabriel César Tavares Ferreira - 21854868
* Maria Luiza Saldanha - 22153140

#**Bibliotecas Utilizadas**

In [22]:
import math
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.stats
from scipy.special import factorial
from tabulate import tabulate

#**BibAD (1o Trabaho)**

In [2]:
def mediaAritmetica(nums):
    return round((sum(nums) / len(nums)), 3)

def var_amostral(nums):
  med = mediaAritmetica(nums)
  quad = np.sum(np.fromiter(((x - med) * (x - med) for x in nums), dtype=float))
  var = quad / len(nums)
  return round(var)

def desvio_padrao(nums):
  return round(math.sqrt(var_amostral(nums)))

def intervaloConfianca(nums, z):
    nivelConfianca = z / 100
    a = 1 - nivelConfianca
    z_a = 1 - (a / 2)
    distribuicao = scipy.stats.norm.ppf(z_a)
    erroPadrao = distribuicao * desvio_padrao(nums) / math.sqrt(len(nums))

    return [round((mediaAritmetica(nums) - erroPadrao), 3), round((mediaAritmetica(nums) + erroPadrao), 3)]

#**BibAD (3° Trabalho)**

In [17]:
def va_exponencial(media):
  u = random.random()
  x = -media * math.log (1 - u)
  return round(x,3)

def e_x(taxa_chegada, taxa_servico):
  rho = taxa_chegada / taxa_servico
  E_X = (rho / taxa_servico) / (1 - rho)
  return E_X

print(va_exponencial(1/9)) #nao esquecer de ver quando receber taxa e quando receber media!!!!!

def fila_mm1(taxa_chegada, taxa_servico, clientes_total):
    E_X = e_x(taxa_chegada, taxa_servico)

    tempos_chegada = np.cumsum([va_exponencial(1/taxa_chegada) for _ in range(clientes_total)])
    tempos_servico = [va_exponencial(1/taxa_servico) for _ in range(clientes_total)]

    tempos_espera = np.zeros(clientes_total)
    tempo_livre = 0

    for i in range(clientes_total):
        if tempos_chegada[i] > tempo_livre:
            tempos_espera[i] = 0
            tempo_livre = tempos_chegada[i] + tempos_servico[i]
        else:
            tempos_espera[i] = tempo_livre - tempos_chegada[i]
            tempo_livre += tempos_servico[i]

    X_n = np.mean(tempos_espera)
    confidencia = 0.95
    z = scipy.stats.norm.ppf(1 - (1 - confidencia) / 2)
    erro_padrao = np.std(tempos_espera, ddof=1) / np.sqrt(clientes_total)
    intervalo_confianca = (X_n - z * erro_padrao, X_n + z * erro_padrao)

    return round(X_n, 3)



0.18


#**Parte 1**

$\bar{X}(n)=\frac{\sum_{i=1}^n x_i}{n}$

* Estimar $\bar{X}(n)$, o tempo médio de espera em uma fila M/M/1 ($n=10^3,λ=9,μ=10$).
* Calcular o valor teórico: $E[X]=\rho \frac{1/\mu}{1-\rho}, \; onde \; \rho = \frac{\lambda}{\mu}$
* Computar o viés (bias): $B=\bar{X}(n)−E[X]$.
* Repetir a simulação r vezes (r=30).
* Anotar os resultados em uma tabela.
* Analisar os resultados.

In [18]:
resultados = fila_mm1(9, 10, 10**3)
print(resultados)

0.921


In [19]:
resultados1 = e_x(9, 10)
print(resultados1)

0.9000000000000001


In [20]:
bias = resultados-resultados1
print(bias)

0.020999999999999908


In [36]:
result_t = [["X", "B"]]
for i in range(30):
  aux = []
  result = fila_mm1(9, 10, 10**3)
  bias = result - e_x(9, 10)
  aux.append(result)
  aux.append(bias)
  result_t.append(aux)

tabela = tabulate(result_t, headers="firstrow", tablefmt="grid")
print(tabela)

+-------+--------+
|     X |      B |
+=======+========+
| 2.622 |  1.722 |
+-------+--------+
| 0.845 | -0.055 |
+-------+--------+
| 0.792 | -0.108 |
+-------+--------+
| 0.811 | -0.089 |
+-------+--------+
| 1.216 |  0.316 |
+-------+--------+
| 0.735 | -0.165 |
+-------+--------+
| 0.391 | -0.509 |
+-------+--------+
| 0.871 | -0.029 |
+-------+--------+
| 0.588 | -0.312 |
+-------+--------+
| 0.431 | -0.469 |
+-------+--------+
| 0.737 | -0.163 |
+-------+--------+
| 1.345 |  0.445 |
+-------+--------+
| 0.924 |  0.024 |
+-------+--------+
| 1.218 |  0.318 |
+-------+--------+
| 0.423 | -0.477 |
+-------+--------+
| 0.556 | -0.344 |
+-------+--------+
| 0.704 | -0.196 |
+-------+--------+
| 0.617 | -0.283 |
+-------+--------+
| 1.242 |  0.342 |
+-------+--------+
| 0.427 | -0.473 |
+-------+--------+
| 0.66  | -0.24  |
+-------+--------+
| 0.578 | -0.322 |
+-------+--------+
| 0.542 | -0.358 |
+-------+--------+
| 3.42  |  2.52  |
+-------+--------+
| 0.365 | -0.535 |
+-------+---

Análise

#**Parte 2**

* Simular o tempo médio de espera em uma fila M/M/1 (λ=9,μ=10).
* Eliminar o transiente usando as heurísticas de (a) Conway e (b) Fishman.
* Após eliminar o transiente, coletar n valores no estado estacionário ( n=103).
* Calcular $\bar{X}(n)$, o tempo médio de espera na fila e o viés (B).
* Repetir a simulação r vezes (r=30).
* Mostrar os resultados em uma tabela.
* Analisar os resultados.

In [37]:
resultados = fila_mm1(9, 10, 10**3)
print(resultados)

1.01


#**Parte 3**

* Executar uma simulação de horizonte infinito da fila M/M/1.
* Eliminar o transiente usando a heurística MSER-5Y.
* Estimar o tempo médio de espera na fila e o respectivo intervalo de confiança de 95%.
* A regra de parada é a precisão relativa de 5%.
* Cenário    I:  λ=7 clientes por segundo; μ=10 clientes por segundo.
* Cenário   II:  λ=8 clientes por segundo; μ=10 clientes por segundo.
* Cenário  III:  λ=9 clientes por segundo; μ=10 clientes por segundo.
* Cenário IV:  λ=9,5 clientes por segundo; μ=10 clientes por segundo.
* Plotar um gráfico comparativo dos quatro cenários.
* Plotar uma linha horizontal para o valor teórico esperado em cada cenário.
* Analisar os resultados.